# Machine Learning for Structural Health Monitoring using BEM

In this work, we'll present a Machine Learning (ML) approach to Structural Health Monitoring (SHM), by using data from Boundary Element Method (BEM) simulation of a beam, the algorithm will be able to predict the existence and position of a fracture. This is performed by regression and classification tasks.

## Objectives

The end goal of this project is to provide a ML workbench based on the BEM to predict anomalies in mechanical parts from sensor measurements. This workbench will consist of a numerical model for the construction of the sensor measurements, before setting an experimental methodology. To achieve the main goal, the following intermediary goals are set:

- Build a simulation tool to provide accurate estimates of sensor measurements (temperature, position);
- Build a dataset containing instances with and without anomalies
- Build a ML model to detect anomalies and predict its attributes, such as position and radius

## Methodology

Consider a mechanical part subject to various forces. This part will be modeled as a two-dimensional rectangle with sides lenght $L_1$ and $L_2$. This part is made from some material such as steel or aluminium, so it has an specific mass $\rho$, specific heat $\gamma$ and speed of propagation $c$. The part will be subjected to known conditions of acoustic pressure and temperature at its boundaries. Sensors will be placed on any position over the surface of the part and will not contribute to the physics but instead be modeled as internal points. The anomaly will be modeled as a circular hole in the part, located at any point of the surface.

The sensors may measure acoustic pressure, displacement or temperature. Initially the temperature measurements will be considered, as it's the case with less attributes, once temperature is a scalar. 

Once modeled, the following attributes will be used to create the training and testing datasets:

- Presence of anomaly;
- Position of the anomaly;
- Radius of the anomaly;
- Temperature measured by the sensors.

Note that the position of the sensor is not part of the attributes but changing them will affect other attributes.

### Boundary Element Method Model

The BEM is chosen as the preferred method for this task for the following advantages over other methods:

- Requires no mesh generation;
- Good accuracy for internal points;
- Fast and simple 2D algorithm for small models.

The two-dimensional BEM will be used to simulate a variety of scenarios, parts with and without anomalies, different sensor and anomaly positions, different number of anomalies.

The model will consist of linear constant elements forming a rectangle of sides $L_1$ and $L_2$.

### Machine Learning Tasks

Once the dataset is built, the ML tasks must be considered. For our purposes of providing a ML tool to assist in SHM procedures and online automatic detection of anomalies, three ML tasks seem most adequate: Anomaly Detection, Regression and Classification.

#### Anomaly Detection

This is a unsupervised task in which normal behavior is provided so that any anomaly or departure from the normal behaviour is detected. This is useful when there is a large amount of data or a data stream from which new data corresponding to the correct behavior of the system. 

This is the case for sensor measurements which can provide new data for training and anomaly detection alike.

#### Regression

Another possibility is to assume there is an anomaly and try to predict its radius and position. This task can be accomplished by Linear and Polynomial Regressions, which are supervised ML algorithms.

#### Classification

Once an anomaly is detected, it may be useful to classify this anomaly based on other attributes. Parts with anomalies of radius larger than a certain amount may be deemed inapropriate to maintain in use. This can be achieved by training aa Classification algorithm so a new instance can be categorized based on its attribute to determine if the part is still in working condition.

In [1]:
include("BB/BEM_base.jl")

BEM_base (generic function with 5 methods)

In [3]:
function generate_part(L_x=1,L_y=1)
    """
    Generate the points and element list for linear elements 
    of a rectangular shape.
    """
    # Define the geometry
    POINTS = [1 0 0
          2 L_x 0
          3 L_x L_y
          4 0 L_y];
    SEGMENTS = [1 1 2 0
             2 2 3 0
             3 3 4 0
             4 4 1 0];
    return POINTS, SEGMENTS
end

function add_holes(POINTS, SEGMENTS,L_x=1,L_y=1,r=0.01,n=1)
    """
    Adds n holes of radius r into the part.
    The holes must be centered in a way so that every hole is
    at least a distance of r away from the walls.
    """
    holes = []
    return POINTS, SEGMENTS, holes
end


function create_sensors(POINTS, SEGMENTS,L_x=1,L_y=1,d=[1,0],h=0.01)
    """
    Creates domain points which will represent the sensors.
    The holes must be centered in a way so that every hole is
    at least a distance of r away from the walls.
    """
    POINTS_dom = []
    NPX = 10; # number of domain points on x and y directions
    NPY = 10;
    PONTOS_DOM=const2D.gera_p_in(NPX,NPY,POINTS,SEGMENTS); # generate domain points
    n_pdom = size(PONTOS_DOM,1);
    PONTOS_dom = zeros(n_pdom,3);
    PONTOS_dom[:,2:3] = PONTOS_DOM[:,:];
    for i = 1:n_pdom
        PONTOS_dom[i,1] = i;
    end
    return POINTS_dom
end

function add_boundary_conditions(POINTS, SEGMENTS,L_x=1,L_y=1,holes,values=[1,-1],values_holes=[0])
    """
    # BCSeg =[NE BT V] are the boundary conditions at each segment.          |                                                                                                                                      |                                                                           
    #NE: element number                                                      |                                                                           
    #BT: boundary condition type                                             |                                                                           
    # BT = 0 if the temperature is known                                     |                                                                           
    # BT = 1 if the temperature gradient is known                            |                                                                           
    #V: value of boundary variable                                           |                                                                           
    """
    BCSeg = [1 1 0
         2 0 0
         3 1 0
         4 0 1];
    return BCSeg
end

function generate_mesh(POINTS, SEGMENTS,L_x=1,L_y=1,holes,ne=10,ne_holes=4)
    """
    Adds n holes of radius r into the part.
    The holes must be centered in a way so that every hole is
    at least a distance of r away from the walls.
    """
    MESH = [1 ne
         2 ne
         3 ne
         4 ne];
    NOS_GEO,NOS,ELEM,CDC,normal = const2D.format_dad(POINTS,SEGMENTS,MESH,BCSeg) # Apply the discretization technique and builds the problems matrices for the geometrical points, physical nodes, elements' connectivity and boundary conditions
    return NOS_GEO,NOS,ELEM,CDC,normal
end

function solve_part()
    """
    """
    PONTOS, SEGMENTOS, MALHA, CCSeg, k = dad_0()
    NOS_GEO,NOS,ELEM,CDC = format_dad(PONTOS,SEGMENTOS,MALHA,CCSeg)
    nnos=size(NOS,1);
    println("Número de nós = $nnos")
    npg=8;
    qsi,w = Gauss_Legendre(-1,1,npg) # Gera os pontos e pesos de Gauss
    Tree,child,center_row,diam,inode,ileaf = cluster(NOS[:,2:3],floor(sqrt(2*length(NOS[:,1]))))

    ninterp=4 # Número de pontos de interpolação
    η =.4 # Coeficiente relacionado a admissibilidade dos blocos
    allow=checa_admiss(η,center_row,diam,inode,ileaf) # mostra quais blocos são admissíveis
    block = blocks(Tree,child,allow) # Função que retorna os blocos admissiveis
    hmati,bi =Hinterp(Tree,block,[NOS,NOS_GEO,ELEM,qsi,w,CDC,k],ninterp)
    xi = gmres(vet->matvec(hmati,vet,block,Tree),bi,5,tol=1e-8,maxIter=1000,out=0) 
    T,q=monta_Teq(CDC,xi[1])


    return T,q
end

function generate_db()
    return
end

generate_part (generic function with 3 methods)

In [8]:
using DelimitedFiles


In [7]:
?DelimitedFiles.writedlm

```
writedlm(f, A, delim='\t'; opts)
```

Write `A` (a vector, matrix, or an iterable collection of iterable rows) as text to `f` (either a filename string or an `IO` stream) using the given delimiter `delim` (which defaults to tab, but can be any printable Julia object, typically a `Char` or `AbstractString`).

For example, two vectors `x` and `y` of the same length can be written as two columns of tab-delimited text to `f` by either `writedlm(f, [x y])` or by `writedlm(f, zip(x, y))`.

# Examples

```jldoctest
julia> using DelimitedFiles

julia> x = [1; 2; 3; 4];

julia> y = [5; 6; 7; 8];

julia> open("delim_file.txt", "w") do io
           writedlm(io, [x y])
       end

julia> readdlm("delim_file.txt", '\t', Int, '\n')
4×2 Matrix{Int64}:
 1  5
 2  6
 3  7
 4  8

julia> rm("delim_file.txt")
```
